# Análisis de Importación de Vehículos desde el Portal SAT

In [27]:
import os
import re
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import zipfile
import csv
import pandas as pd

In [ ]:
urlSat = "https://portal.sat.gob.gt/portal/alza-e-importacion-vehiculos/#1510763502681-dff4b62b-fd76"
baseUrl = "https://portal.sat.gob.gt"
rutaSalida = "./datos/enlaces-importacion-2024_2025.txt"
carpetaZips = "./datos/zips/"
carpetaDescomprimidos = "./datos/descomprimidos/"
archivoUnificado = "./datos/data_unificada.csv"


## Código para bajar los enlaces necesarios

Podemos en un futuro solo aplicar el codigo para descarga de descomprimir los archivos dado los enlaces sin hacer la búsqueda.

In [ ]:
def obtenerImportacionesZips(url):
    resp = requests.get(url, timeout=15)
    resp.raise_for_status()

    soup = BeautifulSoup(resp.text, "html.parser")
    patronZip = re.compile(r"\.zip$", re.I)
    patronImportacion = re.compile(r"/importacion-de-vehiculos/", re.I)

    enlaces = set()

    for a in soup.find_all("a", href=True):
        href = a["href"]
        if (
            patronZip.search(href)
            and ("2024" in href or "2025" in href)
            and patronImportacion.search(href)
        ):
            enlaces.add(urljoin(baseUrl, href))

    return sorted(enlaces)

In [ ]:
def guardarEnlacesImportacion(forzar=True):
    if os.path.exists(rutaSalida) and not forzar:
        respuesta = input(f"El archivo '{rutaSalida}' ya existe. ¿Deseas rehacer el proceso? (s/n): ").strip().lower()
        if respuesta != "s":
            print("Proceso cancelado.")
            return

    print("Obteniendo enlaces...")
    enlaces = obtenerImportacionesZips(urlSat)

    os.makedirs(os.path.dirname(rutaSalida), exist_ok=True)

    with open(rutaSalida, "w", encoding="utf-8") as f:
        f.write("\n".join(enlaces))

    print(f"{len(enlaces)} enlaces guardados en {rutaSalida}")

### Hacer WebScrapping

Se puede cambiar el valor **forzar** a true para hacer la descarga automatica de los enlaces (aún si existe el archivo de enlaces en `./data/enlaces-importacion-2024_2025.txt`)

In [ ]:
guardarEnlacesImportacion(forzar=False)

## Código para descomprimir los archivos Zip

In [44]:
def descargarYDescomprimirZips(ejecutar=True):
    """Descarga y descomprime los ZIP listados en rutaSalida si ejecutar=True."""
    if not ejecutar:
        print("Descarga y descompresión omitidas (ejecutar=False).")
        return

    if not os.path.exists(rutaSalida):
        print(f"No existe el archivo de enlaces: {rutaSalida}")
        return

    os.makedirs(carpetaZips, exist_ok=True)
    os.makedirs(carpetaDescomprimidos, exist_ok=True)

    with open(rutaSalida, "r", encoding="utf-8") as f:
        enlaces = [line.strip() for line in f if line.strip()]

    for enlace in enlaces:
        nombreZip = os.path.basename(enlace)
        rutaZip   = os.path.join(carpetaZips, nombreZip)

        # Descarga
        if not os.path.exists(rutaZip):
            print(f"Descargando: {nombreZip}...")
            try:
                r = requests.get(enlace, timeout=30)
                r.raise_for_status()
                with open(rutaZip, "wb") as fzip:
                    fzip.write(r.content)
                print(f"Descargado: {rutaZip}")
            except Exception as e:
                print(f"Error al descargar {enlace}: {e}")
                continue
        else:
            print(f"Ya existe: {rutaZip}")

        # Descompresión
        try:
            with zipfile.ZipFile(rutaZip, 'r') as zip_ref:
                zip_ref.extractall(carpetaDescomprimidos)
                print(f"Descomprimido en: {carpetaDescomprimidos}")
        except zipfile.BadZipFile:
            print(f"Archivo ZIP corrupto o inválido: {rutaZip}")

### Descomprimir Zips

Se puede modificar el campo `ejecutar` a true o false para ejecutar el proceso de descarga y descomprimir archivos.

In [45]:
descargarYDescomprimirZips(ejecutar=True)

Ya existe: ./datos/zips/importacion_de_vehiculos_2024_enero.zip
Descomprimido en: ./datos/descomprimidos/
Ya existe: ./datos/zips/importacion_de_vehiculos_2024_febrero.zip
Descomprimido en: ./datos/descomprimidos/
Ya existe: ./datos/zips/importacion_de_vehiculos_2024_marzo.zip
Descomprimido en: ./datos/descomprimidos/
Ya existe: ./datos/zips/importacion_de_vehiculos_2024_abril.zip
Descomprimido en: ./datos/descomprimidos/
Ya existe: ./datos/zips/importacion_de_vehiculos_2024_mayo.zip
Descomprimido en: ./datos/descomprimidos/
Ya existe: ./datos/zips/importacion_de_vehiculos_2024_junio.zip
Descomprimido en: ./datos/descomprimidos/
Ya existe: ./datos/zips/importacion_de_vehiculos_2024_julio.zip
Descomprimido en: ./datos/descomprimidos/
Ya existe: ./datos/zips/importacion_de_vehiculos_2024_agosto.zip
Descomprimido en: ./datos/descomprimidos/
Ya existe: ./datos/zips/importacion_de_vehiculos_2024_septiembre.zip
Descomprimido en: ./datos/descomprimidos/
Ya existe: ./datos/zips/importacion_de_

## Unificar archivos descomprimidos

In [46]:
def unificarArchivosTxt(ejecutar=True):
    if not ejecutar:
        print("Unificación de archivos omitida (ejecutar=False).")
        return

    archivosTxt = [f for f in os.listdir(carpetaDescomprimidos) if f.endswith(".txt")]

    if not archivosTxt:
        print("No hay archivos .txt en la carpeta descomprimidos.")
        return

    registros = []
    encabezado = None
    archivosLatin1 = []

    for archivo in archivosTxt:
        ruta = os.path.join(carpetaDescomprimidos, archivo)

        # Intentar abrir primero en UTF-8, luego fallback a Latin-1
        try:
            with open(ruta, "r", encoding="utf-8") as f:
                lineas = [line.strip() for line in f if line.strip()]
        except UnicodeDecodeError:
            with open(ruta, "r", encoding="latin-1") as f:
                lineas = [line.strip() for line in f if line.strip()]
            archivosLatin1.append(archivo)

        if not lineas:
            continue

        encabezadoArchivo = lineas[0].rstrip("|")
        columnas = encabezadoArchivo.split("|")

        if encabezado is None:
            encabezado = columnas
        elif encabezadoArchivo != "|".join(encabezado):
            print(f"Encabezado inconsistente en: {archivo}, se omitirá.")
            continue

        for linea in lineas[1:]:
            linea = linea.rstrip("|")
            partes = linea.split("|")
            if len(partes) < len(encabezado):
                partes += [""] * (len(encabezado) - len(partes))
            elif len(partes) > len(encabezado):
                partes = partes[:len(encabezado)]
            registros.append(partes)

    if not registros:
        print("No se encontraron registros válidos.")
        return

    # Guardar CSV
    os.makedirs(os.path.dirname(archivoUnificado), exist_ok=True)
    with open(archivoUnificado, "w", newline="", encoding="utf-8") as fcsv:
        writer = csv.writer(fcsv)
        writer.writerow(encabezado)
        writer.writerows(registros)

    print(f"{len(registros)} registros guardados en {archivoUnificado}")

    if archivosLatin1:
        print("\nArchivos abiertos con latin-1 (no eran UTF-8):")
        for nombre in archivosLatin1:
            print(f" - {nombre}")

### Unificar archivos

Se puede modificar el campo `ejecutar` a true o false para ejecutar el proceso unificación de archivos.

In [47]:
unificarArchivosTxt(ejecutar=True)

962721 registros guardados en ./datos/data_unificada.csv

Archivos abiertos con latin-1 (no eran UTF-8):
 - web_imp_08022024.txt
 - web_imp_08032024.txt
 - web_imp_08042025.txt
 - web_imp_08052024.txt
 - web_imp_08062024.txt
 - web_imp_08062025.txt
 - web_imp_08072024.txt
 - web_imp_08082024.txt
 - web_imp_08092024.txt
 - web_imp_08102024.txt
 - web_imp_08112024.txt


## Exploración de datos

In [48]:
def analizarCsvUnificado():
    archivo = "datos/data_unificada.csv"

    if not os.path.exists(archivo):
        print(f"No se encontró el archivo: {archivo}")
        return None

    df = pd.read_csv(archivo, encoding="utf-8")

    # Limpiar nombres de columna (espacios antes/después)
    df.columns = df.columns.str.strip()

    print("Análisis básico del archivo unificado:")
    print(f"- Filas: {df.shape[0]}")
    print(f"- Columnas: {df.shape[1]}")
    print("\nColumnas:")
    print(df.columns.tolist())

    print("\nValores nulos por columna:")
    print(df.isnull().sum())

    total_nulos = df.isnull().sum().sum()
    print(f"\nTotal de valores nulos en todo el DataFrame: {total_nulos}")

    return df

In [49]:
df = analizarCsvUnificado()

Análisis básico del archivo unificado:
- Filas: 962721
- Columnas: 17

Columnas:
['Pais de Proveniencia', 'Aduana de Ingreso', 'Fecha de la Poliza', 'Partida Arancelaria', 'Modelo del Vehiculo', 'Marca', 'Linea', 'Centimetros Cubicos', 'Distintivo', 'Tipo de Vehiculo', 'Tipo de Importador', 'Tipo Combustible', 'Asientos', 'Puertas', 'Tonelaje', 'Valor CIF', 'Impuesto']

Valores nulos por columna:
Pais de Proveniencia        0
Aduana de Ingreso           0
Fecha de la Poliza          0
Partida Arancelaria         0
Modelo del Vehiculo         0
Marca                       0
Linea                       8
Centimetros Cubicos     12255
Distintivo              28704
Tipo de Vehiculo            0
Tipo de Importador          0
Tipo Combustible            0
Asientos                    1
Puertas                     1
Tonelaje                    0
Valor CIF                   0
Impuesto                    0
dtype: int64

Total de valores nulos en todo el DataFrame: 40969


### Análisis para el año 2024

#### ¿Cuántos **vehículos livianos** de **cada tipo** se importaron en 2024?

In [50]:
def conteoLivianosPorTipo(df):
    # Asegura nombres de columnas limpios
    df.columns = df.columns.str.strip()

    # Convierte la fecha a datetime (formato DD/MM/YYYY)
    df["Fecha de la Poliza"] = pd.to_datetime(
        df["Fecha de la Poliza"], dayfirst=True, errors="coerce"
    )

    # Filtra registros 2024 y distintivo “LIVIANO”
    mask = (
        (df["Fecha de la Poliza"].dt.year == 2024) &
        (df["Distintivo"].str.upper() == "LIVIANO")
    )

    # Agrupa por tipo de vehículo y cuenta
    return (
        df.loc[mask]
          .groupby("Tipo de Vehiculo")
          .size()
          .reset_index(name="Total")
          .sort_values("Total", ascending=False)
          .reset_index(drop=True)
    )
conteo = conteoLivianosPorTipo(df)
print(conteo)


      Tipo de Vehiculo   Total
0                 MOTO  428341
1            CAMIONETA   58901
2              PICK UP   41112
3            AUTOMOVIL   33422
4              TRIMOTO   11861
5             MICROBUS    2082
6         CAMIONETILLA    1771
7                PANEL    1325
8                 JEEP    1074
9     VEHICULO RUSTICO     616
10     CAMIONETA SPORT     610
11  CAMIONETA AGRICOLA     117
12             MINIBUS      12
13       CARRO FUNEBRE       4
14        MINI TRACTOR       3
15    CARRETA-CARRETON       2


#### ¿Cuál es la **distribución de modelos** de **carros**, **pickups** y **SUV** importados en ese año?

#### ¿Cuál es el **tipo de vehículo** que más se importó durante el 2024?

#### ¿Cuáles fueron los **meses con mayor importación** de vehículos livianos?

### Comparativo 2024 vs 2025

#### ¿Cómo vamos con la importación de **cada tipo de vehículo** en los primeros meses de 2025 comparado con los mismos meses de 2024?